# Part 4: Predictions on Separate Dataset

In this final part, I use the most successful random forest classifier model from the previous part to test a different set of test data.

## 1. Setting Up Training Data

I first set up all of the data from the previous parts as the training data.

In [1]:
import numpy as np
import pandas as pd

In [15]:
df_list = []
for i in range(0,12):
    filename = 'playlist' + str(i) + '.json'
    df_list.append(pd.read_json(filename))

songs = pd.concat(df_list)
songs = songs.reset_index()
songs = songs.drop_duplicates()
songs = songs.drop('index', axis=1)
songs = songs.drop(['id', 'title'], axis=1)

In [16]:
songs.head()

danceability  energy  key  loudness  mode  acousticness  instrumentalness  \
0         0.521   0.673    8    -8.685     1      0.005730          0.000000   
1         0.735   0.849    4    -4.308     0      0.212000          0.000029   
2         0.614   0.547    5   -11.772     0      0.767000          0.893000   
3         0.769   0.817    0    -4.092     0      0.026800          0.000099   
4         0.623   0.793   11    -6.630     0      0.000397          0.001500   

   liveness  valence    tempo  duration_ms  time_signature  num_bars  \
0    0.1200    0.543  108.031       225947               4       100   
1    0.0608    0.223  125.972       207477               4       107   
2    0.0919    0.226   92.990       172000               4        65   
3    0.0840    0.866  139.979       218615               4       125   
4    0.3750    0.360   98.998       274213               4       112   

   num_sections  num_segments  class  
0             8           830      1  
1             7           999      1  
2             8           698      1  
3            10           921      1  
4             9          1137      1

## 2. Setting Up Test Data

I then read the test data from the separate test playlists. Note that I use a different file naming format to read the .json files for which I slightly modified the 'extractor.py' script. This is because I ran the script at different times for the training and test data, and the test .json files would overwrite the training .json files.

In [19]:
df_list = []
for i in range(0,2):
    filename = 'playlist_test' + str(i) + '.json'
    df_list.append(pd.read_json(filename))

songs_test = pd.concat(df_list)
songs_test = songs_test.reset_index()
songs_test = songs_test.drop_duplicates()
songs_test = songs_test.drop(['index', 'id'], axis=1) # keeping title to self-check data once predictions are done

In [20]:
songs_test.head()

title  danceability  energy  key  loudness  mode  \
0                 Like Silk         0.748   0.321    7    -9.464     1   
1                Lion Heart         0.754   0.788    2    -3.175     1   
2                Love Alone         0.596   0.817    8    -3.769     1   
3  Love Lies (with Normani)         0.708   0.648    6    -5.626     1   
4            Love Like That         0.761   0.407    2    -6.616     1   

   acousticness  instrumentalness  liveness  valence    tempo  duration_ms  \
0      0.389000          0.001910    0.1070    0.245   76.965       124675   
1      0.129000          0.002740    0.0775    0.961  124.992       224258   
2      0.000192          0.000003    0.0619    0.445  130.927       211647   
3      0.095600          0.000000    0.1340    0.338  143.955       201707   
4      0.343000          0.030600    0.3080    0.854  154.049       214400   

   time_signature  num_bars  num_sections  num_segments  class  
0               4        39             6           506      1  
1               4       115            10           841      1  
2               4       113             8           793      1  
3               4       120            14           747      1  
4               4       136             8           888      1

## 3. Applying Random Forest Classifier Model

Instead of splitting the data into train and split, I used all of the training data to train the model and all of the testing data to test the data.

In [23]:
X_train = songs.drop('class', axis=1)
X_test = songs_test.drop(['class','title'], axis=1)
y_train = songs['class']
y_test = songs_test['class']

In [12]:
from sklearn.metrics import confusion_matrix, classification_report

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [34]:
rfc_pred = rfc_model.predict(X_test)
print(confusion_matrix(y_test, rfc_pred))
print('\n')
print(classification_report(y_test, rfc_pred))

[[37  5]
 [11 31]]


             precision    recall  f1-score   support

          0       0.77      0.88      0.82        42
          1       0.86      0.74      0.79        42

avg / total       0.82      0.81      0.81        84



The results are actually better than those in Part 3, with a noticeable improvement in the precision in predicting class 0. In the parts below, I createa a dataframe with the title, and the actual and predicted classes to look at how the model worked on certain songs.

In [48]:
songs_test_pred = songs_test[['title','class']]
songs_test_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 2 columns):
title    84 non-null object
class    84 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


In [49]:
songs_test_pred['pred_class'] = rfc_pred

C:\Users\Simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
songs_test_pred.sort_values('title')

title  class  pred_class
35                             A Million Miles      1           1
37                                       LOSER      1           1
5                                LOVE ME RIGHT      1           1
7                                LOVE SCENARIO      1           0
42                                  Lead Me On      0           0
43                          Learned From Texas      0           1
54                        Left Side Of Leavin'      0           0
65                                      Let It      0           0
76                              Let It Be Mine      0           0
79             Let It Fly (feat. Travis Scott)      0           1
80                              Letting You Go      0           0
81                                   Leviathan      0           0
82                         Lick (feat. Offset)      0           0
83                                Life Goes On      0           0
44                           Life in Your Days      0           1
46                                   Lifestyle      0           0
45                                   Lifestyle      0           0
47                            Like I Loved You      0           0
0                                    Like Silk      1           0
48                            Lil Bebe - Remix      0           0
1                                   Lion Heart      1           1
49                                    Lit Thot      0           0
50                    Little Things Mean A Lot      0           0
51                              Live Sheck Wes      0           0
52               Live That Life (feat. Garren)      0           0
12                        Locked Out Of Heaven      1           1
53                                 Lonely Call      0           0
56                                 Look At You      0           0
23   Look what you've done (Old Playa reprise)      1           1
58                                     Lose It      0           0
..                                         ...    ...         ...
69                                     Maayera      0           0
70                                  Mad Hatter      0           0
15                                       Magic      1           1
71                     Magic (feat. Tedi Pagg)      0           1
16                                Magic--Pearl      1           1
72                                   Magically      0           0
17                                    Magnolia      1           1
73                     Major (feat. Key Glock)      0           0
74                               Make It Sweet      0           0
75                             Make Me Want To      0           0
18                           Make Up Your Mind      1           0
19                             Makes Me Wonder      1           1
77                               Makin' Me Say      0           0
78                                    Mamacita      0           0
22                                        Maps      1           1
24                                     Marbles      1           1
25                                   Marry You      1           0
26                                   Mauna Kea      1           1
27                                 Me Like Yuh      1           1
28  Me Like Yuh (feat. Hoody) [Korean Version]      1           1
29                                      Melted      1           1
30                                    Memories      1           1
33                           Midsummer Madness      1           1
36                                Millionaires      1           0
55                                The Long Way      0           0
20                  The Man Who Can't Be Moved      1           1
21                  The Man Who Sold The World      1           1
60                                       a lot      0           0
57                                        lose      0           0
41             

## 4. Conclusion

The random forest classifier model seems to be fairly accurate in predicting whether I will like a certain song or not based on the given data. The results could potentially improve with a better estimator implementation, or with better data. One example of having better data would be the way I classify the songs. Instead of having binary classes of whether I simply like or dislike a song, I could rate each song from 1-5 and see if the different classes are clustered within the data points. I would then predict how much I like or dislike a certain song.